# Resnet50 Inference

## Description
This example performs inference on a short wildlife video using a Resnet50 V2 model that has been pre-trained on imagenet data. The labels used for each class are simplified for readability, but still reflect the correct index-label pairs in official use. 

In [1]:
import numpy as np
import cv2
import json
import time
import os.path
from os import path 
import sys

### Importing MIGraphX Library
Sometimes the PYTHONPATH variable is not set during installation of MIGraphX. If your receive a "Module Not Found" error when trying to `import migraphx` in your own application, try running:
```
$ export PYTHONPATH=/opt/rocm/lib:$PYTHONPATH
```
For this example, the library will be added to the kernel's sys.path.

In [2]:
migx_lib_path = "/opt/rocm/lib"
if migx_lib_path not in sys.path:
    sys.path.append(migx_lib_path)

In [3]:
import migraphx

If this is your first time running this example, you will need to dowload the model and sample video.

The following cell will ask you for your sudo password and then install/update the package `youtube-dl` if necessary. It will then use that tool to download the sample video.

In [4]:
if not path.exists("./sample_vid.mp4"):
    import getpass
    import os
    password = getpass.getpass()
    command = "sudo -H -S pip install --upgrade youtube-dl"
    os.system('echo %s | %s' % (password, command))
    !youtube-dl https://youtu.be/TkqYmvH_XVs 
    !mv sample_vid-TkqYmvH_XVs.mp4 sample_vid.mp4

········
[youtube] TkqYmvH_XVs: Downloading webpage
[youtube] TkqYmvH_XVs: Downloading MPD manifest
[dashsegments] Total fragments: 34
[download] Destination: sample_vid-TkqYmvH_XVs.f137.mp4
[download] 100% of 70.35MiB in 00:06.31MiB/s ETA 00:000:11
[dashsegments] Total fragments: 18
[download] Destination: sample_vid-TkqYmvH_XVs.f140.m4a
[download] 100% of 2.58MiB in 00:01.99MiB/s ETA 00:000102
[ffmpeg] Merging formats into "sample_vid-TkqYmvH_XVs.mp4"
Deleting original file sample_vid-TkqYmvH_XVs.f137.mp4 (pass -k to keep)
Deleting original file sample_vid-TkqYmvH_XVs.f140.m4a (pass -k to keep)


The following will download the resnet50 v2 model from ONNX's model zoo.

In [5]:
if not path.exists("./resnet50.onnx"):
    !wget https://github.com/onnx/models/blob/master/vision/classification/resnet/model/resnet50-v2-7.onnx?raw=true
    !mv 'resnet50-v2-7.onnx?raw=true' resnet50.onnx

--2021-01-13 20:24:16--  https://github.com/onnx/models/blob/master/vision/classification/resnet/model/resnet50-v2-7.onnx?raw=true
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/onnx/models/raw/master/vision/classification/resnet/model/resnet50-v2-7.onnx [following]
--2021-01-13 20:24:16--  https://github.com/onnx/models/raw/master/vision/classification/resnet/model/resnet50-v2-7.onnx
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://media.githubusercontent.com/media/onnx/models/master/vision/classification/resnet/model/resnet50-v2-7.onnx [following]
--2021-01-13 20:24:16--  https://media.githubusercontent.com/media/onnx/models/master/vision/classification/resnet/model/resnet50-v2-7.onnx
Resolving media.githubusercontent.com (media.githubusercontent.com)... 151.101.48.133


Load the simplified imagenet labels.

In [6]:
with open('imagenet_simple_labels.json') as json_data:
    labels = json.load(json_data)

### Model and Video Capture Setup

The ONNX graph that is loaded by `parse_onnx()` is a generalized representation that must be compiled for a specific target before it can be executed. For this example, using the target "gpu" is recommended. 

In [7]:
model = migraphx.parse_onnx("resnet50.onnx")
model.compile(migraphx.get_target("gpu"))
model.print()     # Printed in terminal 
cap = cv2.VideoCapture("sample_vid.mp4")

### Pre-Processing Video Frames
Resnet50 requires some preprocessing of video frames before it can run inference. 

The model will expect an NCHW tensor with the shape {1, 3, 224, 224} and the values loaded into a range of [0, 1] and then normalized using mean = [0.485, 0.456, 0.406] and std = [0.229, 0.224, 0.225]. 

The first step is to square up the dimensions of the original image by cropping the longer of the two to the size of the shorter dimension. This will help to avoid any stretching or compressing of the input image.
Then the image can be scaled up or down to the desired resolution of 224x224.

In [8]:
def make_nxn(image, n):
    width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    if height > width:
        dif = height - width
        bar = dif // 2 
        square = image[(bar + (dif % 2)):(height - bar),:]
        return cv2.resize(square, (n, n))
    elif width > height:
        dif = width - height
        bar = dif // 2
        square = image[:,(bar + (dif % 2)):(width - bar)]
        return cv2.resize(square, (n, n))
    else:
        return cv2.resize(image, (n, n))

Now that the image data has the correct dimensions, the values can be normalized as described above.

In [9]:
def preprocess(img_data):
    mean_vec = np.array([0.485, 0.456, 0.406])
    stddev_vec = np.array([0.229, 0.224, 0.225])
    norm_img_data = np.zeros(img_data.shape).astype('float32')
    for i in range(img_data.shape[0]):  
        norm_img_data[i,:,:] = (img_data[i,:,:]/255 - mean_vec[i]) / stddev_vec[i]
    return norm_img_data

### Run Inference on Single Frame

The above pre-processing functions can now be applied to individual video frames and the data can be passed to the model for evaluation. 

In [10]:
def predict_class(frame) -> int:
    # Crop and resize original image
    cropped = make_nxn(frame, 224)
    # Convert from HWC to CHW
    chw = cropped.transpose(2,0,1)
    # Apply normalization
    pp = preprocess(chw)
    # Add singleton dimension (CHW to NCHW)
    data = np.expand_dims(pp.astype('float32'),0)
    # Run the model
    results = model.run({'data':data})
    # Extract the index of the top prediction
    res_npa = np.array(results[0])
    return np.argmax(res_npa)

### Inference Loop over Full Video

Now everything is in place so that we can run inference on each frame of the input video. The video will be played and the predicted label will be displayed on top of each frame.

In [11]:
while (cap.isOpened()):
    start = time.perf_counter()
    ret, frame = cap.read()
    if not ret: break
    
    top_prediction = predict_class(frame)
    
    end = time.perf_counter()
    fps = 1 / (end - start)
    fps_str = f"Frames per second: {fps:0.1f}"
    label_str = "Top prediction: {}".format(labels[top_prediction])

    labeled = cv2.putText(frame, 
                          label_str, 
                          (50, 50), 
                          cv2.FONT_HERSHEY_SIMPLEX, 
                          2, 
                          (255, 255, 255), 
                          3, 
                          cv2.LINE_AA)
    labeled = cv2.putText(labeled, 
                          fps_str, 
                          (50, 1060), 
                          cv2.FONT_HERSHEY_SIMPLEX, 
                          2, 
                          (255, 255, 255), 
                          3, 
                          cv2.LINE_AA)
    cv2.imshow("Resnet50 Inference", labeled)

    if cv2.waitKey(1) & 0xFF == ord('q'): # 'q' to quit
        break

cap.release()
cv2.destroyAllWindows()